# Scraping des notes de plusieurs séries sur Metacritic

In [4]:
from multidl_mieux import *
import urllib.request as urllib2
from bs4 import BeautifulSoup

In [5]:
# les fonctions du démon
def get_headers(url):
    return {'Referer' : url, 'User-Agent' : 'Mozilla/5.0 (X11; U; Linux x86_64; en-US; rv:1.9.1.3)'}

def get_headers2(url):
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    return {'Referer' : url, 'User-Agent' : user_agent}


def get_request(url): 
    return urllib2.Request(url, None, get_headers2(url))

def my_cb(page,code,url,**kwargs): 
    print("%s\n%s\n param sup : %s" % (url,page,str(kwargs)))

def my_cb2(page,code,url,**kwargs):
    print("")

In [7]:
def get_page_code(url, dm) :
    """
    url : string, l'url de la page que l'on veut
    dm : MultiDLManager
    renvoie la page (en bytes) associée à l'url en entrée et le code 
    """
    dm.add_task(get_request(url),my_cb2)
    page, code = dm.get_request(url)
    return page, code


def get_html_soup(page) :
    """
    page : bytes, code html de la page que l'on veut
    renvoie la page sous une forme exploitable avec BeautifulSoup
    """
    # page est en bytes (string avec un 'b' devant)
    response = str(page, 'utf-8') # pour convertir en string classique
    html_soup = BeautifulSoup(response, 'html.parser')
    return html_soup


def get_reviews_from_metacritic_saison(html_soup, nom_serie, nb_reviews=100) :
    """
    html_soup : code html de la page que l'on veut 
    nb_reviews : int, nb de reviews par page
    nom_serie : string, nom de la serie
    renvoie un dictionnaire {username : {serie : note}}
    """
    reviews = html_soup.find_all('div', class_ = 'review_stats')
    if len(reviews) == 0 :
        print('Plus de reviews')
        return -1 # pour savoir quand est-ce qu'on atteint la fin des reviews
    
    d = dict()
    while len(reviews) != 0 :
        for i in range(0, nb_reviews) :
            name = reviews[i].find('div', class_='name')
            user = list(name.children)[1].text # nom de l'utilisateur
            r = reviews[i].find('div', class_="review_grade")
            rating = int(r.div.text) # sa note pour la saison 
            d[user] = {nom_serie : rating}
        
    return d

In [3]:
def get_url_metacritic(nom_serie, nb_saison, nb_page) :
    url = "https://www.metacritic.com/tv/"+nom_serie+"/season-"+str(nb_saison)+"/user-reviews?page="+str(nb_page)
    return url

In [20]:
def get_reviews_from_metacritic(nom_serie, dm,  nb_total_saisons , nb_reviews=100) :
    """
    html_soup : code html de la page que l'on veut 
    nb_reviews : int, nb de reviews par page
    nom_serie : string, nom de la serie
    nb_total_saisons : int, nb totals de saisons de cette série
    renvoie un dictionnaire {username : {serie : list(note)}}
    """

    d = dict()
    for nb_saison in range(1, nb_total_saisons+1) :
        nb_page = 0
        while True :
            print("Saison "+str(nb_saison)+" page "+str(nb_page))
            url = get_url_metacritic(nom_serie, nb_saison,nb_page)

            page, code = get_page_code(url, dm)
            html_soup = get_html_soup(page)

            reviews = html_soup.find_all('div', class_ = 'review_stats')
            
            nb_reviews = len(reviews) - 3
            
            if nb_reviews == 0 :
                print('\tPlus aucune review pour '+nom_serie+" saison "+str(nb_saison)+" page "+str(nb_page))
                break

            for i in range(0, nb_reviews) :
                name = reviews[i].find('div', class_='name')
                user = list(name.children)[1].text # nom de l'utilisateur
                r = reviews[i].find('div', class_="review_grade")
                rating = int(r.div.text) # sa note pour la saison 

                if nb_saison == 1 or user not in d.keys():
                    d[user] = {nom_serie : [rating]}
                else  :
                    d[user][nom_serie].append(rating)
                    
            nb_page += 1
                    
       
    return d

## Toutes les saisons de Dexter

In [9]:
dm = MultiDLManager()

DEBUG:stem:System call: tor --version (runtime: 0.02)


In [21]:
dexter = get_reviews_from_metacritic("dexter", dm,  8 , nb_reviews=100)

Saison 1 page 0

Saison 1 page 1
Saison 1 page 2


Saison 1 page 3
Saison 1 page 4


Saison 1 page 5

Plus aucune review pour dexter saison 1 page 5
Saison 2 page 0
Saison 2 page 1
Plus aucune review pour dexter saison 2 page 1
Saison 3 page 0



Saison 3 page 1

Plus aucune review pour dexter saison 3 page 1
Saison 4 page 0

Saison 4 page 1

Plus aucune review pour dexter saison 4 page 1
Saison 5 page 0
Saison 5 page 1


Plus aucune review pour dexter saison 5 page 1
Saison 6 page 0

Saison 6 page 1

Plus aucune review pour dexter saison 6 page 1
Saison 7 page 0
Saison 7 page 1


Plus aucune review pour dexter saison 7 page 1
Saison 8 page 0

Saison 8 page 1

Plus aucune review pour dexter saison 8 page 1


In [22]:
keys = dexter.keys()

In [24]:
len(keys) 

657

In [25]:
dexter

{'4za': {'dexter': [10]},
 'AD': {'dexter': [10]},
 'AJSearles': {'dexter': [10]},
 'AS': {'dexter': [9]},
 'Abutreca': {'dexter': [2]},
 'Adam': {'dexter': [10]},
 'AdamG': {'dexter': [10]},
 'AdamL': {'dexter': [10]},
 'AdamS': {'dexter': [10]},
 'AdrianM': {'dexter': [10]},
 'AlS.': {'dexter': [10]},
 'AlanM': {'dexter': [10]},
 'AlexBugella': {'dexter': [10]},
 'AlexK': {'dexter': [10]},
 'AlexN.': {'dexter': [10]},
 'Alexandre': {'dexter': [10]},
 'AlexisK': {'dexter': [10]},
 'AlphaBoss': {'dexter': [9]},
 'AlyA': {'dexter': [10]},
 'AmandaA.': {'dexter': [10]},
 'Amazin': {'dexter': [10]},
 'AmyM': {'dexter': [10]},
 'AndreaW': {'dexter': [10]},
 'Andremax': {'dexter': [8, 6]},
 'AndrewS': {'dexter': [10]},
 'AndréL': {'dexter': [10]},
 'AngeloO.': {'dexter': [10]},
 'AnnS': {'dexter': [10]},
 'AnnaC': {'dexter': [9]},
 'AnnaT': {'dexter': [10]},
 'AnnetteR': {'dexter': [10]},
 'AnthonyP.': {'dexter': [10]},
 'ArmandoJ': {'dexter': [10]},
 'ArmchairS.': {'dexter': [0]},
 'Armote

La plupart des utilisateurs n'ont noté qu'une seule saison et on dirait que la plupart des gens qui ont noté cette série l'ont aimée.

On pourrait faire un fichier avec les noms de toutes les séries de notre base (en enlevant les nombres devant et les underscores) et en les transformants pour les faire coller au maximum avec la convention du site (minuscules, tiret entre les mots, ...). Il faudrait faire ça et voir combien de séries de notre on peut récupérer comme ça pour voir si c'est faisable ou pas. Pour l'instant le problème avec cette idée c'est que le code du dm fait une erreur quand on essaye d'accéder à une page qui fait une erreur 404 (l'idéal, ça serait juste de passer à la page suivante). Le but :  
`
cpt = 0
for nom_serie in liste_noms :
    url = "https://www.metacritic.com/tv/"+nom_serie
    page, code = get_page_code(url, dm)
    if code != 404 :
        cpt +=1
`
pour savoir combien de séries on peut exploiter with minimum effort.


- le fichier c'est facile, la transformation un peu moins straightforward mais pas techniquement compliquée
- c'est surtout de changer le code pour pouvoir exploiter les erreurs 404 qui m'embêtte
- après c'est du tout cuit
    
    
Si jamais peu de séries exploitables de cette façon (moins de 1000), il faudra regarder IMDB avec http://www.omdbapi.com/ pour les ID des séries. Sur IMDB, les reviews sur pas sur des pages différentes, c'est avec un read-more donc il faudra voir comment faire pour régler ça aussi. 
